In [1]:
import os
import pandas as pd


In [2]:
# locate all csv files in directory
directory = '../Assets/Datasets/BTS_flight_data/Raw'

file_names = []
for filename in os.listdir(directory):
    if filename.endswith(".csv"): 
        file_names.append(os.path.join(directory, filename))


In [3]:
# combine all csvs in pandas
dataframes = []
ct = 1
files = len(file_names)
for csv in file_names:
    dataframes.append(pd.read_csv(csv))
    print 'Added file {} of {}'.format(ct, files)
    ct += 1

df = pd.concat(dataframes)

Added file 1 of 67
Added file 2 of 67
Added file 3 of 67
Added file 4 of 67
Added file 5 of 67
Added file 6 of 67
Added file 7 of 67
Added file 8 of 67
Added file 9 of 67
Added file 10 of 67
Added file 11 of 67
Added file 12 of 67
Added file 13 of 67
Added file 14 of 67
Added file 15 of 67
Added file 16 of 67
Added file 17 of 67
Added file 18 of 67
Added file 19 of 67
Added file 20 of 67
Added file 21 of 67
Added file 22 of 67
Added file 23 of 67
Added file 24 of 67
Added file 25 of 67
Added file 26 of 67
Added file 27 of 67
Added file 28 of 67
Added file 29 of 67
Added file 30 of 67
Added file 31 of 67
Added file 32 of 67
Added file 33 of 67
Added file 34 of 67
Added file 35 of 67
Added file 36 of 67
Added file 37 of 67
Added file 38 of 67
Added file 39 of 67
Added file 40 of 67
Added file 41 of 67
Added file 42 of 67
Added file 43 of 67
Added file 44 of 67
Added file 45 of 67
Added file 46 of 67
Added file 47 of 67
Added file 48 of 67
Added file 49 of 67
Added file 50 of 67
Added fil

/Users/mwitebsky/anaconda2/envs/lesson01/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Added file 59 of 67
Added file 60 of 67
Added file 61 of 67
Added file 62 of 67
Added file 63 of 67
Added file 64 of 67
Added file 65 of 67
Added file 66 of 67
Added file 67 of 67


In [4]:
# scheduled Departure/Arrival time for ohare and midway
# takes a few min
crs_time = lambda row:  (row['CRS_DEP_TIME'] if (row['ORIGIN_AIRPORT_ID'] == 13930) | 
                        (row['ORIGIN_AIRPORT_ID'] == 13232) 
                        else row['CRS_ARR_TIME'])

df.loc[:, 'CRS_FL_TIME'] =  df.apply(crs_time, axis=1)

In [5]:
# actual departure/arrival time ohare and midway
# takes a few min
flight_time = lambda row: (row['DEP_TIME'] if (row['ORIGIN_AIRPORT_ID'] == 13930) |
                          (row['ORIGIN_AIRPORT_ID'] == 13232) 
                          else row['ARR_TIME'])

df.loc[:, 'FL_TIME'] =  df.apply(flight_time, axis=1)

In [6]:
df['FL_DATE'] = pd.to_datetime(df['FL_DATE'], infer_datetime_format=True)

In [7]:
df = df.sort_values(['TAIL_NUM', 'FL_DATE', 'FL_TIME'])

In [9]:
# Remove null TAIL_NUM rows
df.dropna(subset=['TAIL_NUM'], inplace=True)
df.reset_index(inplace=True)
df.drop('index', axis=1, inplace=True)

In [11]:
df_2 = df[['ORIGIN_AIRPORT_ID', 'DEST_AIRPORT_ID', 'TAIL_NUM']]

In [12]:
df_2.loc[:, 'PRIOR_DEST_ID'] = df_2['DEST_AIRPORT_ID'].shift()
df_2.loc[:, 'PRIOR_ORIG_ID'] = df_2['ORIGIN_AIRPORT_ID'].shift()
df_2.loc[:, 'PRIOR_TAIL_NUM'] = df_2['TAIL_NUM'].shift()
df_2.loc[:, 'PRIOR_AIRPORT'] = 0

In [13]:
# Add PRIOR_AIRPORT column
# takes a few min
prior_airport = lambda row: (row['PRIOR_ORIG_ID'] 
                             if ((row['ORIGIN_AIRPORT_ID'] == 13930) | (row['ORIGIN_AIRPORT_ID'] == 13232)) &
                                (row['PRIOR_DEST_ID'] == row['ORIGIN_AIRPORT_ID']) &
                                (row['TAIL_NUM'] == row['PRIOR_TAIL_NUM'])
                             else 0)

df_2.loc[:, 'PRIOR_AIRPORT'] = df_2.apply(prior_airport, axis=1)
    

In [14]:
df_pa = pd.DataFrame(df_2['PRIOR_AIRPORT'])
df = df.join(df_pa, how='left')

In [15]:
df.to_csv('../Assets/Datasets/BTS_flight_data/Processed/Illinois_2011_2016_with_prior.csv', index=False)

In [2]:
df = pd.read_csv('../Assets/Datasets/BTS_flight_data/Processed/Illinois_2011_2016_with_prior.csv')

In [16]:
# Departures from MDW & ORD only
df = df[df['ORIGIN_AIRPORT_ID'].isin([13930, 13232])]
df.shape

(2155102, 38)

In [17]:
# Create MDW & ORD departures
df.to_csv('../Assets/Datasets/BTS_flight_data/Processed/chicago_departures.csv', index=False)

In [18]:
# Create cancelled dataframe and save as csv
cancelled = df[df['CANCELLED'] == 1.0]
cancelled.to_csv('../Assets/Datasets/BTS_flight_data/Processed/cancelled_chicago_departures.csv', index=False)

In [19]:
# create departures dataframe  without cancellations
departures = df[df['CANCELLED'] == 0.0]
departures.to_csv('../Assets/Datasets/BTS_flight_data/Processed/non_cancelled_chicago_departures.csv', index=False)